# Dashboard Understanding


***Data Flow***

--> Select Points $(x, Q^2)$ 

--> add noise to cross section ***(created from concatenation of obs_p and obs_n of corresponding $(x,Q^2)$)***

--> send the crossection data to the Model 

--> generates output (parameters) 

--> Parameters are used to generate PDF UP and PDF DOWN 


***Equation to add Noise***

$(xsec)' = xsec + (\sigma * f * R)$

- uncertainity ($\sigma$)

- noise ($f$)

- Random value ($R$)

***To generate PDFs***

$x$ = an array of range(0,100)

$Q^2$ = constant value

In [1]:
# libraries
import numpy as np #--- pdf,mlutils
import pandas as pd
import plotly.graph_objs as go
import dash
import matplotlib.pyplot as plt

#---mlutils
from tensorflow.keras.models import load_model
#from keras.models import load_model
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import h5py

#---pdf libs
from scipy.special import gamma
import sys,os

#--- only for jupyter
from ipywidgets import interactive, HBox, VBox

In [2]:
#-- Loading the data
X = np.load("data/X.npy")
Q2 = np.load("data/Q2.npy")
obs_p = np.load("data/obs_p.npy") #-- toy data is of shape (100,101)
obs_n = np.load("data/obs_n.npy") #-- toy data is of shape (100,101)
obs_p.shape, obs_n.shape, X.shape, Q2.shape

((100, 101), (100, 101), (101,), (101,))

In [3]:
data = pd.DataFrame(data = X,
                    index=np.arange(len(X)),
                    columns=['X']
                    )
data['Q2'] = Q2
data['obs_p'] = obs_p[0,:] #--- currently taking only one sample
data['obs_n'] = obs_n[0,:] #--- currently taking only one sample
data['is_selected'] = False
data['co-ord'] = list(zip(X,Q2))
data.head()

,X,Q2,obs_p,obs_n,is_selected,co-ord
0,0.012055,1.000000,2.627395,0.727704,False,"(0.012055362337673993, 1.0)"
1,0.019467,1.000000,2.862549,0.816804,False,"(0.019466967101596316, 1.0)"
2,0.019467,1.260221,2.798435,0.799198,False,"(0.019466967101596316, 1.260220615891982)"
3,0.019467,1.588156,2.739728,0.783056,False,"(0.019466967101596316, 1.5881560007191666)"
4,0.031435,1.000000,3.108782,0.920222,False,"(0.031435206800076304, 1.0)"


In [4]:
dataSelected = False
print(dataSelected)
def selectedData(trace,points,selector):
    global dataSelected
    dataSelected = True
    #print(points)
    temp_x = []
    temp_q2 = []
    temp_op = []
    temp_on = []
    t.data[0].cells.values = [data.loc[points.point_inds][col] for col in ['X','Q2','obs_p','obs_n']]
    data.loc[points.point_inds,'is_selected'] =  True
    return dataSelected

False


In [5]:
#plotting the data
figure = go.FigureWidget(data=go.Scattergl(x=data['X'],y=data['Q2'],mode='markers',),
                   layout=go.Layout(xaxis_title="X",yaxis_title="Q2"))
#figure.show()
input_data = figure.data[0]

#--- for jupyter notebook
# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(
    header=dict(values=['X','Q2','obs_p','obs_n'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[data[col] for col in ['X','Q2','obs_p','obs_n']],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])


input_data.on_selection(selectedData)


# Put everything together ---- jupyter
VBox((figure,t))

    'data': [{'mode': 'markers',
              'type': 'scattergl',
            …

In [6]:
print(dataSelected)
data[data['is_selected'] == True]

True


,X,Q2,obs_p,obs_n,is_selected,co-ord
81,0.9,1.000000,0.356107,0.093364,True,"(0.9, 1.0)"
82,0.9,1.260221,0.354355,0.092944,True,"(0.9, 1.260220615891982)"
83,0.9,1.588156,0.352701,0.092546,True,"(0.9, 1.5881560007191666)"
84,0.9,2.001427,0.351136,0.092169,True,"(0.9, 2.0014269333588555)"
85,0.9,2.522239,0.349649,0.091810,True,"(0.9, 2.5222394826202974)"
86,0.9,3.178578,0.348233,0.091467,True,"(0.9, 3.1785781942148255)"
87,0.9,4.005710,0.346883,0.091140,True,"(0.9, 4.005709769574232)"
88,0.9,5.048078,0.345591,0.090827,True,"(0.9, 5.048078032897367)"
89,0.9,6.361692,0.344353,0.090526,True,"(0.9, 6.361692007688706)"
90,0.9,8.017135,0.343165,0.090237,True,"(0.9, 8.017135420044562)"


#### Constant Values For Uncertainity, Noise (F-value)


In [7]:
uncertainity_value = 0.05
f_value = 5

#--- default values of noise and uncertainity
d_uncertainity_value = 0.05
d_f_value = 1

#### Adding noise to the crossectional values

we add noise repeatedly for a range of 1000 where we get an array of shape (1000,101)

In [12]:
def addNoiseSelected(data, n_value, uncertainity_value, validation_column, data_column, num_features):
    predList = []
    #--- formula 
    #--- for the selected_data : xsec + Randomnumber * uncertainity * noise_value
    for i in range(1000):
        predList.append(np.where(data[validation_column]==True, #--- if data is selected
                                 data[data_column] + (float(uncertainity_value) * np.random.randn(num_features,) * float(n_value)), #---- add value (noise * uncertainity * random number)
                                 data[data_column] + (float(d_uncertainity_value) * np.random.randn(num_features,) * float(d_f_value)))) #--- else remain the data as it is
        #predList.append(data + (alpha * np.random.rand(num_features,)))
    predList = np.array(predList)
    return predList


In [13]:
def calculate_xsec(p_Data, n_Data):
    return np.concatenate((p_Data, n_Data), axis = 1)

In [14]:
#--- if data is not selected we add noise to all the values
if not dataSelected : 
    data['is_selected'] = True
    print('--> no data selected so adding noise to every point')

#--- get the list of data with added noise
obs_p_noised = addNoiseSelected(data, f_value, uncertainity_value, 'is_selected', 'obs_p', len(data['obs_p']))
obs_n_noised = addNoiseSelected(data, f_value, uncertainity_value, 'is_selected', 'obs_n', len(data['obs_n']))

#--- concatenate the lists to get the cross-sectional Data
xsec = calculate_xsec(obs_p_noised, obs_n_noised)
print(xsec.shape)

(1000, 202)


In [15]:
print(data['obs_p'].to_numpy())
print(obs_p_noised[0])

[2.62739496 2.86254862 2.79843463 2.73972796 3.10878211 3.04933402
 2.99471389 2.9442627  2.8974438  3.35659226 3.30344547 3.25444417
 3.20903486 3.16676522 3.12726121 3.09021019 3.58513279 3.54019841
 3.49861805 3.45995439 3.42384878 3.39000366 3.3581695  3.32813513
 3.29972031 3.74999402 3.71536454 3.6831932  3.65316802 3.62503212
 3.59857158 3.57360641 3.54998376 3.52757277 3.50626056 3.48594912
 3.75762651 3.73516719 3.71420144 3.69454633 3.67605045 3.65858711
 3.64204921 3.62634543 3.61139726 3.59713674 3.58350463 3.57044898
 3.55792403 3.41611362 3.40619255 3.39685002 3.38802057 3.37964942
 3.37169021 3.36410333 3.35685472 3.34991481 3.34325782 3.33686107
 3.33070454 3.32477049 3.31904304 3.31350802 2.37396808 2.37246489
 2.37097214 2.36949452 2.36803518 2.36659621 2.36517894 2.36378415
 2.36241224 2.36106331 2.35973725 2.35843382 2.35715266 2.35589334
 2.35465539 2.3534383  2.35224153 0.35610736 0.35435511 0.35270143
 0.35113571 0.34964895 0.34823348 0.3468827  0.34559092 0.3443

In [16]:
print('--> Before Adding Noise : ',data['obs_p'].to_numpy()[-3:])
print('--> After Adding Noise : ',obs_p_noised[0][-3:])

--> Before Adding Noise :  [0.33506127 0.33418759 0.33333884]
--> After Adding Noise :  [0.34282126 0.18783542 0.62385435]


In [17]:
print('--> Before Adding Noise : ',data['obs_n'].to_numpy()[-3:])
print('--> After Adding Noise : ',obs_n_noised[0][-3:])

--> Before Adding Noise :  [0.08825534 0.08804079 0.0878322 ]
--> After Adding Noise :  [0.21578853 0.16168074 0.1892816 ]


#### Running the model to generate the parameters

In [18]:
def backwardPredict(fname, model, xsec_noised):
    # load xsec file
    #xsec = calculate_xsec(dataframe)
    ml = load_model('mldata/%s.h5'%model) 

    # make the prediction
    pred = ml.predict(xsec_noised)
    #pred = par_scaler.inverse_transform(pred)
    np.save('data/%s-par.npy'%fname, pred)
    return pred

In [19]:
fname = 'test_backward'
model_select = 'my_model'
nn_pred = backwardPredict(fname, model_select, xsec)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
nn_pred.shape

(1000, 10)

#### Calculate pdfs from generated parmaeters

In [21]:
#--- need to know the significance of each term
Q02=4.0
lam2=0.2**2
pu=np.ones(10)
pd=np.ones(10)
pu[5:]*=0.1
pd[5:]*=0.1

In [22]:
def set_params(par):
    pu[:5]=par[:5]
    pd[:5]=par[5:]
  
def get_s(Q2):
    return np.log(np.log(Q2/lam2)/np.log(Q02/lam2))

def _get_shape(x,p):
    return p[0]*x**p[1]*(1-x)**p[2]*(1+p[3]*x+p[4]*x**2)
    
def get_shape(x,p,s):
    N=p[0] + p[5] * s
    a=p[1] + p[6] * s
    b=p[2] + p[7] * s
    c=p[3] + p[8] * s
    d=p[4] + p[9] * s
    return _get_shape(x,[N,a,b,c,d])

def get_pdf(x,Q2,flav):
    s=get_s(Q2)
    if flav=='u': return get_shape(x,pu,s)
    if flav=='d': return get_shape(x,pd,s)
    
def calculate_pdf(data_par):
    x = np.linspace(0.01,0.99,100)
    Q2=4.0
    u=[]
    d=[]
    for i in range(data_par.shape[0]):
        set_params(data_par[i])
        u.append(get_pdf(x,Q2,'u'))
        d.append(get_pdf(x,Q2,'d'))
    u = np.array(u)
    d = np.array(d)
    print('--> (RUNLOG) - Up data shape : ',u.shape)
    print('--> (RUNLOG) - Down data Shape : ',d.shape)
    return {'u' : u, 'd' : d, 'x-axis' : x}

In [23]:
pdf_dict = calculate_pdf(nn_pred)

--> (RUNLOG) - Up data shape :  (1000, 100)
--> (RUNLOG) - Down data Shape :  (1000, 100)


In [25]:
#--- Plot the figure (Ratio Plot)
figure_g = go.Figure()
pdf_up_trace = go.Scattergl(
                        x = pdf_dict['x-axis'],
                        y = np.ones(len(pdf_dict['x-axis'])),#pdf_dict['u'].mean(axis=0),
                        name='PDF UP',
                        showlegend=True,
                        error_y=dict(
                                type='data',
                                color='orange',
                                array=pdf_dict['u'].std(axis=0)/pdf_dict['d'].mean(axis=0),
                                #array=pdf_dict['u'].std(axis=0)*5,
                                visible=True
                                )
                        )
#--- Plotting pdf_up
figure_g.add_trace(pdf_up_trace)